# Layer Fidelity实验

LF相比QV聚焦于量子芯片中质量最好的量子比特，LF提供了一种可扩展、更全面的平均性能测量。LF实验的理论如下图所示：

![LF_theory.png](images/LF_theory.png)

# 理论
在参考文献[1]中介绍了LF实验的步骤，具体如下：

1. 选择量子比特链
2. 将量子比特链拆分成若干不相交的层
3. 对每一层分别运行1Q/2Q的SDRB实验，得到每一层的层保真度($LF_m$)
4. 将每一层的层保真度相乘得到整体的层保真度($LF$),最后计算得到EPLG (error per layered gate)，计算方式如下:
$$EPLG = 1 - LF^{1/n_{2q}}$$

这里的$n_{2q}$表示双门数量，即量子比特数减1.

在开始LF实验之前，我们要导入一些包以及确定LF实验的配置参数，才能正确运行LF实验，这些配置参数包括：用到的量子比特链、LF实验的层数、每层的量子线路数量、线路提交地址等。

LF实验的总体代码流程如下：

In [ ]:
# 导入必要的包
import os
import json
import pathlib
from originbench.LF.lf import LFExperiment
from originbench.LF.lf_options import *
from originbench.LF.my_mapping import gen_qubit_mapping_from_line
from originbench.LF.drb import DRBExperiment
from originbench.LF.lf_storage import (
    LF_RESULT_FILE, LF_CIRCUIT_ENTRY, N_2Q_ENTRY, FIDELITY_ENTRY, LF_ENTRY,
    EPLG_ENTRY, DRB_RESULT_FILES_ENTRY,
)

# 确定LF实验的配置参数
def save_lf_options(
    options: LFOptions,
    file: str=LF_OPTION_FILE,
) -> None:
    working_dir = options.get_option(WORKING_DIR_ENTRY)
    if not os.path.exists(working_dir):
        os.makedirs(working_dir)

    s = str(pathlib.Path(os.getcwd()) / file)
    print(f'Saving LF options to {s}')
    options.save_options(filename=file)


# 量子比特链
q_line = list(range(41, 36, -1))
dev_type = 'originq'

# LF实验的层数
layers = [i+1 for i in range(6)]

# 每层的量子线路数量
circs_per_layer = 1
shots = 10**3

# 用户的API key
api_key_qcloud = ("your api token")
api_key = api_key_qcloud

is_mapping = False
is_amend = True
is_optimization = False

dir_name = 'qcloud'
working_dir = str(pathlib.Path(os.getcwd()) / 'test' / 'LF' / dir_name)
print(f'Working directory: {working_dir}')

auto_clean = True
is_cloud_simulation = False

lf_option = LFOptions(
    working_dir=working_dir,
    qubit_line=q_line,
    layers=layers,
    circs_per_layer=circs_per_layer,
    shots=shots,
    is_mapping=is_mapping,
    is_amend=is_amend,
    is_optimization=is_optimization,
    using_qcloud=True,
    dev_type=dev_type,
    submit_url=None,
    api_key=api_key,
    chip_id=72,
    auto_clean=auto_clean,
    is_cloud_simulation=is_cloud_simulation,
)

save_lf_options(lf_option)

实验配置设置好了之后，我们可以开始LF实验，量子霸权实验主要由以下几个步骤组成：
1. 构建量子线路
2. 提交量子线路
3. 获取量子线路结果
4. 结果处理

下面我们分别来介绍上述几个步骤。

## 1. 构建量子线路
这一步中，我们需要根据量子比特链、LF实验层数、量子线路shots次数以及每层量子线路的数量来构建LF实验量子线路。

In [ ]:
def generate_lf_circuits(options: LFOptions):
    """Generate all LF quantum circuits.
    """
    qubit_line = options[QUBIT_LINE_ENTRY]
    layers = options[LAYERS_ENTRY]
    samples = options[SHOTS_ENTRY]
    circs_per_layer = options[CIRCS_PER_DEPTH_ENTRY]


    mapping = gen_qubit_mapping_from_line(qubit_line)
    n_qubits = len(qubit_line)

    print(
        f'Number of qubits: {n_qubits}, '
        f'Qubit mapping: {mapping}'
    )

    # Construct an LFExperiment object, this will generate all quantum circuits
    # and save quantum circuits into files, the files names will be stored in
    # `LF_RESULT_FILE`.
    lf_experiment = LFExperiment(
        n_qubits=n_qubits,
        layers=layers,
        samples=samples,
        circs_per_layer=circs_per_layer,
        mapping=mapping,
    )

    lf_experiment.build_drbs()


original_dir = os.getcwd()
print(f'current folder: {original_dir}')
working_dir = lf_option[WORKING_DIR_ENTRY]
print(f'Working directory: {working_dir}')
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

os.chdir(working_dir)
print('Generating LF circuits...')
generate_lf_circuits(options=lf_option)
print('Generating LF circuits done.')

## 2. 提交量子线路
量子线路生成之后，我们就可以将它提交给我们的量子计算机运行。

In [ ]:
def submit_lf_circuis(option: LFOptions) -> None:
    """Run LF circuits.
    """
    dev_type = option[DEV_TYPE_ENTRY]
    submit_url = option[SUBMIT_URL_ENTRY]
    api_key = option[API_KEY_ENTRY]
    auto_mapping = option[IS_MAPPING_ENTRY]
    is_amend = option[IS_AMEND_ENTRY]
    using_qcloud = option[USING_QCLOUD_ENTRY]
    chip_id = option[CHIP_ID_ENTRY]

    with open(LF_RESULT_FILE, 'r') as f:
        data = json.load(f)
    lf_circuit_file = data[LF_CIRCUIT_ENTRY]

    drb_result_files = []
    for circuit_file in lf_circuit_file:
        drb_result_file_i = DRBExperiment.run_all_layer_circuits_from_file(
            file=circuit_file,
            submit_url=submit_url,
            api_key=api_key,
            auto_mapping=auto_mapping,
            is_amend=is_amend,
            chip_id=chip_id,
            dev_type=dev_type,
            using_qcloud=using_qcloud,
        )
        print(f'Adding drb result file: {drb_result_file_i}')
        drb_result_files.append(drb_result_file_i)
    
    print(f'Save result file into {LF_RESULT_FILE}')
    data[DRB_RESULT_FILES_ENTRY] = drb_result_files
    with open(LF_RESULT_FILE, 'w') as f:
        json.dump(data, f, indent=2)

    
print('Submitting LF circuits...')
submit_lf_circuis(option=lf_option)
print('Submit LF circuits done.')

## 3. 获取量子线路结果
在获取量子线路结果时，往往会遇到量子线路没有计算完成，我们通常可以等待芯片结果计算完成后一次性获取结果。

In [ ]:
def update_lf_result(option: LFOptions):
    """Update LF experiment results.
    """
    dev_type = option[DEV_TYPE_ENTRY]
    submit_url = option[SUBMIT_URL_ENTRY]
    api_key = option[API_KEY_ENTRY]
    using_qcloud = option[USING_QCLOUD_ENTRY]

    with open(LF_RESULT_FILE, 'r') as f:
        data = json.load(f)
    drb_result_files = data[DRB_RESULT_FILES_ENTRY]
    for result_file_i in drb_result_files:
        print(f'Update result for result file: {result_file_i}')
        DRBExperiment.update_result_file(
            file=result_file_i,
            dev_type=dev_type,
            submit_url=submit_url,
            api_key=api_key,
            using_qcloud=using_qcloud,
        )


print('Updating LF results...')
update_lf_result(option=lf_option)
print('Update LF results done.')

## 4. 结果处理

当我们获得了所有量子线路的芯片运行结果，必须等上一步任务查询出现结果了方可进行下一步结果分析，数据分析过程如下：

In [ ]:
def process_lf_result():
    """Process LF results.
    """
    with open(LF_RESULT_FILE, 'r') as f:
        data = json.load(f)
    drb_result_files = data[DRB_RESULT_FILES_ENTRY]
    n_2q = data[N_2Q_ENTRY]
    lf = 1.0

    fidelities = []
    for result_file_i in drb_result_files:
        fid_i = DRBExperiment.process_result_file(
            file=result_file_i,
            show_fig=False,
        )
        lf *= fid_i
        fidelities.append(fid_i)
    eplg = 1 - lf**(1/n_2q)

    data[FIDELITY_ENTRY] = fidelities
    data[LF_ENTRY] = lf
    data[EPLG_ENTRY] = eplg

    with open(LF_RESULT_FILE, 'w') as f:
        json.dump(data, f, indent=2)

    print('='*200)
    print(f'LF={lf}, EPLG={eplg}')
    print('='*200)


print('Processing LF results...')
process_lf_result()

from datetime import datetime
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print('Process LF results done : ',current_time)

# Do not forget to change back to the original directory.
os.chdir(original_dir)

# 参考文献
[1] David C. McKay et al. Benchmarking Quantum Processor Performance at Scale. arXiv:2311.05933 [quant-ph] (2023).